In [1]:
import grequests as asynhttp
import pandas as pd
import json

### Data Collection and Cleaning

In [2]:
capitales = {
    "Africa": ["El Cairo", "Abuya", "Nairobi", "Addis Abeba", "Pretoria"],
    "America": ["Washington D. C.", "Ottawa", "Ciudad de México", "Brasilia", "Buenos Aires",],
    "Asia": ["Pekín", "Tokio", "Nueva Delhi", "Seúl", "Riad"],
    "Europa": ["Madrid", "París", "Berlín", "Roma", "Londres"],
    "Oceania": ["Canberra", "Wellington", "Port Moresby", "Suva", "Apia"],
}
key="030c86ad50fe0b9ad9817deeb4fc2825"

In [3]:
localizaciones={
    "name": [],
    "lat": [],
    "lon": []
}

rs=[]
for continente in capitales:
    for ciudad in capitales[continente]:
        url=f"http://api.openweathermap.org/geo/1.0/direct?q={ciudad}&appid={key}"
        rs.append(asynhttp.get(url))
        
futures = asynhttp.map(rs)
for futuro in futures:
    response=futuro.json()
    localizaciones["name"].append(response[0]["name"])
    localizaciones["lat"].append(response[0]["lat"])
    localizaciones["lon"].append(response[0]["lon"])

In [4]:
df_localizaciones=pd.DataFrame(localizaciones).set_index("name")
df_localizaciones

,lat,lon
name,,
Cairo,30.044388,31.235726
Abuja,9.064331,7.489297
Nairobi,-1.303264,36.826384
Addis Ababa,9.010793,38.761252
Pretoria,-25.745928,28.187910
Washington,38.895037,-77.036543
Ottawa,45.420878,-75.690111
Mexico City,19.432630,-99.133178
Brasília,-15.793404,-47.882317


In [5]:
rs=[]
for entrada in df_localizaciones.values:
    url=f"https://api.openweathermap.org/data/2.5/weather?lat={entrada[0]}&lon={entrada[1]}&appid={key}&units=metric&lang=sp"
    rs.append(asynhttp.get(url))

tiempo=[]
futures = asynhttp.map(rs)
for futuro in futures:
    tiempo.append(json.loads(futuro.text))

In [6]:
for ciudad in tiempo:
    del ciudad["coord"]
    for i, e in enumerate(ciudad["weather"]):
        del ciudad["weather"][i]["id"]
        del ciudad["weather"][i]["icon"]
    del ciudad["sys"]
    del ciudad["timezone"]
    del ciudad["id"]
    del ciudad["name"]
    del ciudad["cod"]

In [7]:
df_tiempo = pd.json_normalize(tiempo, "weather", ["base", ["main", "feels_like"],
    ["main", "temp_min"],["main", "temp_max"],["main", "pressure"],["main", "humidity"],
    ["main", "sea_level"],["main", "grnd_level"], "visibility", ["wind", "speed"], ["wind", "deg"], 
    ["clouds", "all"], "dt"], sep="-")

In [8]:
df_tiempo = df_localizaciones.reset_index().join(df_tiempo)

In [9]:
df_tiempo.head()

,name,lat,lon,main,description,base,main-feels_like,main-temp_min,main-temp_max,main-pressure,main-humidity,main-sea_level,main-grnd_level,visibility,wind-speed,wind-deg,clouds-all,dt
0,Cairo,30.044388,31.235726,Clear,cielo claro,stations,19.78,20.14,20.33,1019,52,1019,1013,10000,4.63,30,0,1764695571
1,Abuja,9.064331,7.489297,Clouds,algo de nubes,stations,29.55,31.07,31.07,1006,27,1006,954,10000,1.38,155,23,1764695546
2,Nairobi,-1.303264,36.826384,Clouds,nubes dispersas,stations,19.14,19.75,19.75,1014,52,1014,838,10000,3.6,50,40,1764695571
3,Addis Ababa,9.010793,38.761252,Clouds,nubes dispersas,stations,12.18,13.54,13.54,1017,47,1017,775,10000,2.75,72,28,1764695546
4,Pretoria,-25.745928,28.187910,Rain,lluvia ligera,stations,17.76,17.93,17.93,1012,76,1012,867,10000,6.26,189,81,1764695327


In [18]:
df_tiempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             25 non-null     object 
 1   lat              25 non-null     float64
 2   lon              25 non-null     float64
 3   main             25 non-null     object 
 4   description      25 non-null     object 
 5   base             25 non-null     object 
 6   main-feels_like  25 non-null     object 
 7   main-temp_min    25 non-null     object 
 8   main-temp_max    25 non-null     object 
 9   main-pressure    25 non-null     object 
 10  main-humidity    25 non-null     object 
 11  main-sea_level   25 non-null     object 
 12  main-grnd_level  25 non-null     object 
 13  visibility       25 non-null     object 
 14  wind-speed       25 non-null     object 
 15  wind-deg         25 non-null     object 
 16  clouds-all       25 non-null     object 
 17  dt               2

### Data Analysis
- Which cities are the hottest and coldest right now?
- Is there a relationship between temperature and humidity?
- Which continent seems to have the windiest cities?
- What’s the most common weather description (e.g., clear, cloudy, rainy)?